In [ ]:
import numpy as np
import pandas as pd
from zipfile import ZipFile

zip_file = ZipFile('archive.zip')
data_hrStp = pd.read_csv(zip_file.open('Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv'), index_col=1)
data_hrCal = pd.read_csv(zip_file.open('Fitabase Data 4.12.16-5.12.16/hourlyCalories_merged.csv'), index_col=1)
data_mnSlp = pd.read_csv(zip_file.open('Fitabase Data 4.12.16-5.12.16/minuteSleep_merged.csv'), index_col=1)
data_mnInt = pd.read_csv(zip_file.open('Fitabase Data 4.12.16-5.12.16/minuteIntensitiesWide_merged.csv'), index_col=1)

# Convert index to datetime (needed for resampling):
data_hrStp.index = pd.to_datetime(data_hrStp.index)
data_hrCal.index = pd.to_datetime(data_hrCal.index)
data_mnSlp.index = pd.to_datetime(data_mnSlp.index)
data_mnInt.index = pd.to_datetime(data_mnInt.index)


In [ ]:
## CONVERT minute Sleep file to hourly:

# Add separated columns for sleep quality: Per dictionary: 1 = asleep, 2 = restless, 3 = awake
data_mnSlp['1=asleep'] = data_mnSlp['value'].apply(lambda x: 1 if x == 1 else 0)
data_mnSlp['2=restless'] = data_mnSlp['value'].apply(lambda x: 1 if x == 2 else 0)
data_mnSlp['3=awake'] = data_mnSlp['value'].apply(lambda x: 1 if x == 3 else 0)
print('Orig data with added columns:\n', data_mnSlp.head())
print()

# Resample minute Sleep data into hourly Sleep data, keeping date, Id and 3 columns for sleep quality
#data_hrSlp = data_mnSlp.groupby('Id', as_index=False)[['Id','1=asleep','2=restless','3=awake']].resample('1H').mean().reset_index()
data_hrSlp = data_mnSlp[['Id','1=asleep','2=restless','3=awake']]
data_hrSlp = data_hrSlp.groupby('Id', as_index=False).resample('1H').agg({'Id':'last','1=asleep':'sum','2=restless':'sum','3=awake':'sum'}).reset_index()

# Reset table to match other dataframes
data_hrSlp.set_index('date', inplace=True)
data_hrSlp.index.name = 'ActivityHour'
data_hrSlp.drop(columns=['level_0'], inplace=True)
data_hrSlp.dropna(subset=['Id'], inplace=True)
data_hrSlp = data_hrSlp.astype({'Id':'int64'})

#print(data_hrSlp.info())
print('Resampled data:\n', data_hrSlp.head())


Orig data with added columns:
                              Id  value        logId  1=asleep  2=restless  \
date                                                                        
2016-04-12 02:47:30  1503960366      3  11380564589         0           0   
2016-04-12 02:48:30  1503960366      2  11380564589         0           1   
2016-04-12 02:49:30  1503960366      1  11380564589         1           0   
2016-04-12 02:50:30  1503960366      1  11380564589         1           0   
2016-04-12 02:51:30  1503960366      1  11380564589         1           0   

                     3=awake  
date                          
2016-04-12 02:47:30        1  
2016-04-12 02:48:30        0  
2016-04-12 02:49:30        0  
2016-04-12 02:50:30        0  
2016-04-12 02:51:30        0  

Resampled data:
                              Id  1=asleep  2=restless  3=awake
ActivityHour                                                  
2016-04-12 02:00:00  1503960366         5           4        4
2016-

In [ ]:
## SUMMARIZE minute Intensities file to hourly:

data_mnInt.index = pd.to_datetime(data_mnInt.index)

# Add separated columns for intensity level: Per dictionary: 0 = Sedentary, 1 = Light, 2 = Moderate, 3 = Very Active
data_mnInt['0=sedentary'] = data_mnInt.eq(0).sum(axis=1)
data_mnInt['1=light'] = data_mnInt.eq(1).sum(axis=1)
data_mnInt['2=moderate'] = data_mnInt.eq(2).sum(axis=1)
data_mnInt['3=veryactive'] = data_mnInt.eq(3).sum(axis=1)

# Summarize by id/hour:
data_hrInt = data_mnInt[['Id','0=sedentary','1=light','2=moderate','3=veryactive']]
print('Summarized data:\n', data_hrInt.head(5))


Summarized data:
                              Id  0=sedentary  1=light  2=moderate  \
ActivityHour                                                        
2016-04-13 00:00:00  1503960366           46       14           0   
2016-04-13 01:00:00  1503960366           60        0           0   
2016-04-13 02:00:00  1503960366           60        0           0   
2016-04-13 03:00:00  1503960366           56        4           0   
2016-04-13 04:00:00  1503960366           60        0           0   

                     3=veryactive  
ActivityHour                       
2016-04-13 00:00:00             0  
2016-04-13 01:00:00             0  
2016-04-13 02:00:00             0  
2016-04-13 03:00:00             0  
2016-04-13 04:00:00             0  


In [ ]:
# Counts:
print(data_hrStp.info())
print(data_hrCal.info())
print(data_hrInt.info())
print(data_hrSlp.info())
print()

# Merge 4 dataframes:
merged_df = pd.merge(data_hrStp, data_hrCal, on=["Id", "ActivityHour"], how="inner")
merged_df = pd.merge(merged_df, data_hrInt, on=["Id", "ActivityHour"], how="outer")
merged_df = pd.merge(merged_df, data_hrSlp, on=["Id", "ActivityHour"], how="outer")
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22099 entries, 2016-04-12 00:00:00 to 2016-05-12 14:00:00
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Id         22099 non-null  int64
 1   StepTotal  22099 non-null  int64
dtypes: int64(2)
memory usage: 517.9 KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22099 entries, 2016-04-12 00:00:00 to 2016-05-12 14:00:00
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Id        22099 non-null  int64
 1   Calories  22099 non-null  int64
dtypes: int64(2)
memory usage: 517.9 KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21645 entries, 2016-04-13 00:00:00 to 2016-05-13 07:00:00
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Id            21645 non-null  int64
 1   0=sedentary   21645 non-null  int64
 2   1=light     

In [ ]:
# Final cleanup:
merged_df.fillna(0, inplace=True)
merged_df = merged_df.astype(int)  #convert back to int64; can only be done after dropping NAs
merged_df.sort_values(['Id','ActivityHour'], inplace=True)

print(merged_df.head())
print(merged_df.tail())

#export to excel for group assignment
merged_df.to_excel("hourlyActivity_merged.xlsx", sheet_name='hourlyActivity')


                             Id  StepTotal  Calories  0=sedentary  1=light  \
ActivityHour                                                                 
2016-04-12 00:00:00  1503960366        373        81            0        0   
2016-04-12 01:00:00  1503960366        160        61            0        0   
2016-04-12 02:00:00  1503960366        151        59            0        0   
2016-04-12 03:00:00  1503960366          0        47            0        0   
2016-04-12 04:00:00  1503960366          0        48            0        0   

                     2=moderate  3=veryactive  1=asleep  2=restless  3=awake  
ActivityHour                                                                  
2016-04-12 00:00:00           0             0         0           0        0  
2016-04-12 01:00:00           0             0         0           0        0  
2016-04-12 02:00:00           0             0         5           4        4  
2016-04-12 03:00:00           0             0        57   